In [1]:
#Importing libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from tensorflow import keras
import PIL
from PIL import Image
import os, sys

Using TensorFlow backend.


In [ ]:
###Use this if you need to create images of the same type & size###
##This section is NOT needed to run the model everytime

#Path of directory
train_path = "More Fake/"
dirs = os.listdir( path1 )

#Resizing images to be 200x200 px
def resize_img(path):
    dirs2 = os.listdir(path)
    for item in dirs2:
        if item == '.DS_Store':
            continue
        img = Image.open(path+item)
        f, e = os.path.splitext(path+item)
        imResize = img.resize((200, 200), Image.ANTIALIAS)
        imResize.save(f + 'new_image.png', 'PNG')

#Running the function
resize_img(path1) 

In [37]:
#Creating paths to the training and testing data
train_data_dir = 'Feature & Real Train'
validation_data_dir = 'Feature & Real Test'

train_real_dir = 'Feature & Real Train'
train_fake_dir = 'Feature & Real Train'

test_real_dir = 'Feature & Real Test'
test_fake_dir = 'Feature & Real Test'

#counting files in directory
nb_train_samples = len(next(os.walk(train_real_dir))[2] + next(os.walk(train_fake_dir))[2]) - 2
nb_validation_samples = len(next(os.walk(test_real_dir))[2] + next(os.walk(test_fake_dir))[2]) - 2

#Establishing # of epochs and batch size
#I'm using limited epochs and a smaller batch size due to limited computing power
epochs = 5
batch_size = 20



In [38]:
#Stating img dimensions & data format for the model and image generation
img_width, img_height = 200, 200
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [32]:
#Creating CNN
model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), input_shape = input_shape),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(32, (3,3)),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.MaxPooling2D(pool_size = (2, 2)),
    #keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D(64, (3,3)),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(64, (3,3)),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.MaxPooling2D(pool_size = (2, 2)),
    #keras.layers.Dropout(0.3),
    
    keras.layers.Flatten(),
    
    keras.layers.Dense(512),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(14),
    #Using softmax since categorial & output nodes > 2
    keras.layers.Activation('softmax')
])

In [39]:
#augmentation configuration used for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#augmentation configuration used for testing:
#rescaling the images
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 790 images belonging to 14 classes.
Found 129 images belonging to 14 classes.


In [40]:
#now let's compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

test_gen = ImageDataGenerator()

In [42]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    shuffle = True)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 40 steps, validate for 7 steps
Epoch 1/5
40/40 [==============================] - 208s 5s/step - loss: 0.2713 - accuracy: 0.9221 - val_loss: 0.2636 - val_accuracy: 0.9286
Epoch 2/5
40/40 [==============================] - 175s 4s/step - loss: 0.2018 - accuracy: 0.9324 - val_loss: 0.2577 - val_accuracy: 0.9291
Epoch 3/5
40/40 [==============================] - 167s 4s/step - loss: 0.1714 - accuracy: 0.9373 - val_loss: 0.2950 - val_accuracy: 0.9175
Epoch 4/5
40/40 [==============================] - 175s 4s/step - loss: 0.1553 - accuracy: 0.9434 - val_loss: 0.2586 - val_accuracy: 0.9286
Epoch 5/5
40/40 [==============================] - 167s 4s/step - loss: 0.1362 - accuracy: 0.9457 - val_loss: 0.3182 - val_accuracy: 0.9131


In [43]:
nb_samples = len(validation_generator)
model.evaluate_generator(validation_generator, steps=nb_samples)

Instructions for updating:
Please use Model.evaluate, which supports generators.
  ...
    to  
  ['...']


[0.31816257749285015, 0.9130675]

In [59]:
print(model.metrics_names)
###NOTES###

##5 epochs with batch size = 20
## 94.8% accuracy
##test set produced 91% accuracy!!

['loss', 'accuracy']
